In [7]:
import json
with open("hotpot_dev_distractor_v1.json") as f:
    datasets = json.load(f)

In [8]:
datasets = datasets[:500]

In [5]:
from dotenv import load_dotenv
from openai import OpenAI
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def llm_create(prompt):
        messages = [{"role": "user", "content": prompt}]
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )
        return completion.choices[0].message.content

In [14]:
answer_prompt = """Base on the following document and answer the question.
Only answer, no explanation.

Question:{question}

Document:{docs}
"""

In [3]:
from long_memory.component import WeaviateLongMemory

long_mem = WeaviateLongMemory(user="Hotpot", time_sort=False)

Detect existed Hotpot user group memory space, loading...
Detect existed Hotpot user child memory space, loading...


In [11]:
import pandas as pd

df = pd.DataFrame()
df['question_id'] = None
df['answer'] = None
df['org_res_from_long_mem'] = None

In [1]:
import pandas as pd
df = pd.read_json("hotpot_df_v2.json", lines=True)

In [2]:
other_instruct = "Multiple inquiries to ensure the accuracy of evidence"

In [ ]:
error_list = []
for i in range(141, 200):
    try:
        print(f"---{i}---")
        long_mem.del_memory()
        doc_list = datasets[i]["context"]
        question_id = datasets[i]["_id"]
        df.loc[i, 'question_id'] = question_id 
        question = datasets[i]["question"]
        for doc in doc_list:
            article = []
            for content in doc[1]:
                content = content.replace('"', "").strip(".'")
                if content=='' or content=='<br>':
                    pass
                else:
                    article.append({"text":content})
            long_mem.add_article(article)
        res = long_mem.get_memory(question, recall=True, other_instruct=other_instruct)
        df.loc[i, 'org_res_from_long_mem'] = str(res)
        answer_res = llm_create(answer_prompt.format(question=question, docs=res))
        df.loc[i, 'answer'] = answer_res
    except Exception as e:
        print(f"-----error{e}-----")
        error_list.append(i)
        
    if (i+1) % 10 == 0:
        df.to_json("hotpot_df_v2.json", orient="records", lines=True)
df.to_json("hotpot_df_v2.json", orient="records", lines=True)

In [11]:
error_list

[]

In [12]:
df.to_json("hotpot_df_v2.json", orient="records", lines=True)

In [17]:
error_list = []
for i in range(200):
    try:
        print(f"---{i}---")
        long_mem.del_memory()
        doc_list = datasets[i]["context"]
        question_id = datasets[i]["_id"]
        df.loc[i, 'question_id'] = question_id 
        question = datasets[i]["question"]
        for doc in doc_list:
            article = []
            for content in doc[1]:
                content = content.replace('"', "").strip(".'")
                if content=='' or content=='<br>':
                    pass
                else:
                    article.append({"text":content})
            long_mem.add_article(article)
        res = long_mem.get_memory(question, recall=True, other_instruct=other_instruct)
        df.loc[i, 'org_res_from_long_mem'] = str(res)
        answer_res = llm_create(answer_prompt.format(question=question, docs=res))
        df.loc[i, 'answer'] = answer_res
    except Exception as e:
        print(f"-----error{e}-----")
        error_list.append(i)
        
    if (i+1) % 10 == 0:
        df.to_json("hotpot_df_v2.json", orient="records", lines=True)
df.to_json("hotpot_df_v2.json", orient="records", lines=True)

,question_id,answer,org_res_from_long_mem
110,5a818387554299260e20a291,Blake Shelton,"{'search times': 1, 'used queries': ['Based on..."
111,5ab978855542996be2020512,The document does not specify the exact number...,"{'search times': 2, 'used queries': ['Over how..."
112,5ae63dad55429929b0807afe,7 January 1936.,"{'search times': 3, 'used queries': ['When was..."
113,5a7a455c5542994f819ef1a7,George Archainbaud died first.,"{'search times': 1, 'used queries': ['Who died..."
114,5a7dcc775542990b8f503ab5,Dinitrophenol,"{'search times': 2, 'used queries': ['Which co..."
115,5a887bc0554299206df2b2b6,Roberto De Vicenzo has won over 230 tournament...,"{'search times': 1, 'used queries': ['The Arge..."
116,5a83880e554299123d8c214e,Marvel Comics,"{'search times': 1, 'used queries': [""Suicide'..."
117,5a879adb5542996e4f30887f,U.S. Representative for Maryland's 8th congres...,"{'search times': 1, 'used queries': ['What oth..."
118,5add11935542994734353827,"No, only Skin Yard is from the U.S.; Ostava is...","{'search times': 1, 'used queries': ['Were the..."
119,5a8c5569554299240d9c2126,Yes.,"{'search times': 1, 'used queries': ['Are Dary..."
